In [3]:
import cv2
import numpy as np

# --- Step 1: Manually select 4 points from a frame ---
def get_points_from_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        raise FileNotFoundError("❌ Could not read frame from video")
    
    points = []

    def click_event(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            print(f"Point selected: ({x}, {y})")
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow("Select 4 Points", frame)

    cv2.imshow("Select 4 Points", frame)
    cv2.setMouseCallback("Select 4 Points", click_event)

    print("👉 Click 4 points in order: Top-Left → Top-Right → Bottom-Right → Bottom-Left")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if len(points) != 4:
        raise ValueError("❌ You must select exactly 4 points.")
    
    return np.float32(points)


# --- Step 2: Crop using perspective transform ---
def crop_video_with_points(input_path, output_path, points):
    cap = cv2.VideoCapture(input_path)
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Destination rectangle size
    width = int(max(np.linalg.norm(points[0] - points[1]), np.linalg.norm(points[2] - points[3])))
    height = int(max(np.linalg.norm(points[0] - points[3]), np.linalg.norm(points[1] - points[2])))

    dst_pts = np.float32([[0, 0], [width-1, 0], [width-1, height-1], [0, height-1]])

    # Compute transform matrix
    M = cv2.getPerspectiveTransform(points, dst_pts)

    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        warped = cv2.warpPerspective(frame, M, (width, height))
        out.write(warped)

    cap.release()
    out.release()
    print("✅ Cropped video saved at:", output_path)


# ----------------------
# USAGE
# ----------------------
video_in = "E:/tennis_detector/.venv/Tennis - 1.mp4"
cropped_out = "cropped_input.mp4"




In [6]:
selected_points = get_points_from_frame(video_in)

👉 Click 4 points in order: Top-Left → Top-Right → Bottom-Right → Bottom-Left


ValueError: ❌ You must select exactly 4 points.